In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import re
from collections import Counter
from tensorflow.keras.layers import TimeDistributed

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
urlOriginalSentence='/content/drive/My Drive/Colab Notebooks/Work/Dataset/original'
urlTargetSentence='/content/drive/My Drive/Colab Notebooks/Work/Dataset/target'

In [0]:
def loadDataset(url,intInd,lastInd):
    sentences=[]
    with open(url,'r') as f:
        sentences=f.read().split('\n')[intInd:lastInd]
    return sentences

In [0]:
startInd=0
lastInd=285000
originalSentence=loadDataset(urlOriginalSentence,startInd,lastInd)
targetSentence=loadDataset(urlTargetSentence,startInd,lastInd)

originalEvalSentence=loadDataset(urlOriginalSentence,lastInd,lastInd+5000)
targetEvalSentence=loadDataset(urlTargetSentence,lastInd,lastInd+5000)

In [0]:
originalSentence=originalSentence[6:]

In [0]:
targetSentence=targetSentence[6:]

In [9]:
vocabulary_size=12000
max_length_original=50 
max_length_target=50
print(max_length_original)
print(max_length_target)

50
50


In [0]:
def cleanOriginalSentence(sentences,wordList=None, addHeaders=False,max_length=None):
    cleanedSentences=[]
    for sentence in sentences:
        sentence=re.sub('[.,\'\"?~!#@$%^&*()]+',"",sentence.lower())
        sentence=re.sub("[ ]+"," ",sentence)
        sentence=re.sub('[^a-zA-Z]+'," ",sentence)
        if len(sentence.split(" "))>max_length-2:
            sentence=" ".join(sentence.split(" ")[:max_length-2])
        if wordList is not None:
            temp=[]
            for word in sentence.split(" "):
                if word in wordList:
                    temp.append(word)
                else:
                    temp.append('unk')
            if len(temp)>max_length-2:
                temp=temp[:max_length-2]
            sentence=" ".join(temp)
        sentence=' '.join([w for w in sentence.split(' ') ])
        if addHeaders==True:
            sentence='<start> ' + sentence+ ' <end>'

        cleanedSentences.append(sentence)
    return cleanedSentences

In [0]:
def cleanTargetSentence(sentences,wordList=None,addHeaders=False,max_length=None):
    cleanedSentences=[]
    for sentence in sentences:
        sentence=re.sub('[.,\'\"?~!#@$%^&*()]+',"",sentence.lower())
        sentence=re.sub("[ ]+"," ",sentence)
        sentence=re.sub('[^а-яА-Я]+'," ",sentence) 
        if len(sentence.split(" "))>max_length-2:
            sentence=" ".join(sentence.split(" ")[:max_length-2])
            
        if wordList is not None:
            temp=[]
            for word in sentence.split(" "):
                if word in wordList:
                    temp.append(word)
                else:
                    temp.append('unk')
            if len(temp)>max_length-2:
                temp=temp[:max_length-2]
            sentence=" ".join(temp)
            
        sentence=' '.join([w for w in sentence.split(" ")])
        if addHeaders==True:
            sentence='<start> ' + sentence+ ' <end>'

        cleanedSentences.append(sentence)
    return cleanedSentences

In [12]:
originalSentence=cleanOriginalSentence(originalSentence,max_length=max_length_original)
counterOriginal=Counter(" ".join(originalSentence).split(" ")).most_common(vocabulary_size)
print(counterOriginal[-1])
counterOriginal={x:y for x,y in counterOriginal}
print(len(counterOriginal))

targetSentence=cleanTargetSentence(targetSentence,max_length=max_length_target)
counterTarget=Counter(" ".join(targetSentence).split(" ")).most_common(vocabulary_size)
counterTarget={x:y for x,y in counterTarget}
print(len(counterTarget))

('printouts', 10)
12000
12000


In [0]:
originalSentence=cleanOriginalSentence(originalSentence,counterOriginal.keys(),True,max_length_original)
originalEvalSentence=cleanOriginalSentence(originalEvalSentence,counterOriginal.keys(),True,max_length_original)

In [0]:
targetSentence=cleanTargetSentence(targetSentence,counterTarget.keys(),True,max_length_target)
targetEvalSentence=cleanTargetSentence(targetEvalSentence,counterTarget.keys(),True,max_length_target)

In [0]:
def tokenize(sentences,max_length):
    tokenizer=keras.preprocessing.text.Tokenizer(filters="",)
    tokenizer.fit_on_texts(sentences)
    tensor=tokenizer.texts_to_sequences(sentences)
    tensor=keras.preprocessing.sequence.pad_sequences(tensor,padding='post',maxlen=max_length)
    return tensor,tokenizer

In [0]:
original,originalTokenizer=tokenize(originalSentence,max_length_original)
target,targetTokenizer=tokenize(targetSentence,max_length_target)


In [0]:

originalEval=keras.preprocessing.sequence.pad_sequences(originalTokenizer.texts_to_sequences(originalEvalSentence),maxlen=max_length_original,padding='post')

targetEval=keras.preprocessing.sequence.pad_sequences(targetTokenizer.texts_to_sequences(targetEvalSentence),maxlen=max_length_target,padding='post')

In [19]:
print(len(originalTokenizer.index_word))
print(len(targetTokenizer.index_word))

12002
12002


In [20]:
len(originalSentence)

284994

In [0]:
batch_size=32
steps_per_epoch=len(original)//batch_size
embedding_dims=300
units=300
vocab_original_size=len(originalTokenizer.index_word)+1
vocab_target_size=len(targetTokenizer.index_word)+1

In [0]:
def generator(encoderInp,decoderInp,max_length,batch_size,vocab_size):
    X1,X2,y=[],[],[]
    n=0
    while 1:
        for enc,dec in zip(encoderInp,decoderInp):
            n+=1
            for i in range(1,len(dec)):
                in_seq,out_seq=dec[:i],dec[i]
                in_seq=keras.preprocessing.sequence.pad_sequences([in_seq],maxlen=max_length)[0]
                out_seq=keras.utils.to_categorical([out_seq],num_classes=vocab_size)[0]
                X1.append(enc[::-1])
                X2.append(in_seq)
                y.append(out_seq)
            if n==batch_size:
                n=0
                yield ([np.array(X1),np.array(X2)],np.array(y))
                X1,X2,y=[],[],[]


In [0]:
train_generator=generator(original,target,max_length_target,batch_size,vocab_target_size)
val_generator=generator(originalEval,targetEval,max_length_target,batch_size,vocab_target_size)

In [24]:
inputEncoder=keras.Input((max_length_original,),name='encoderInput')

embedEnc=keras.layers.Embedding(vocab_original_size,embedding_dims,name='Encoder_Embedding')
gruEnc=keras.layers.GRU(units,return_sequences=True,return_state=True,dropout=0.25,recurrent_dropout=0.25,name='Encoder_GRU1')
se1=embedEnc(inputEncoder)
se2_out,se2_hidden=gruEnc(se1) 
'''
se2_out shape= (batch_size,max_length,units)
se2_hidden shape= (batch_size,units)
'''

inputDecoder=keras.Input((max_length_target,),name='decoderInput')
embedDec=keras.layers.Embedding(vocab_target_size,embedding_dims,name='Decoder_Embedding')
gruDec=keras.layers.GRU(units,dropout=0.25,recurrent_dropout=0.25,return_sequences=True,return_state=True,name='Decoder_GRU1') ## try with statefull
denseDec=keras.layers.Dense(vocab_target_size,activation='softmax')
sd1=embedDec(inputDecoder)
sd2_out,sd2_hidden,=gruDec(sd1,initial_state=[se2_hidden])
sd3=denseDec(sd2_out[:,-1])
model=keras.Model([inputEncoder,inputDecoder],sd3)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
encoder_model=keras.Model(inputEncoder,[se2_hidden])

decoder_state_inp=keras.Input((units,))
decoder_out,decoder_state_out,=gruDec(sd1,initial_state=[decoder_state_inp])
eval_out=denseDec(decoder_out[:,-1])
decoder_model=keras.Model([decoder_state_inp,inputDecoder],[eval_out,decoder_state_out])


In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])


In [26]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2020-02-25 04:18:08--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-02-25 04:18:09--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-02-25 04:18:09--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
with open('/content/glove.6B.300d.txt') as f:
    lines=f.read()

In [0]:
embedding_index={}
for line in lines.split('\n'):
    values=line.split(" ")
    word=values[0]
    values=np.asarray(values[1:],dtype='float64')
    if word in originalTokenizer.word_index.keys():
        embedding_index[word]=values
    

In [0]:
embedding_matrix=np.zeros((vocab_original_size,embedding_dims),dtype='float64')

for word,i in originalTokenizer.word_index.items():
    vec=embedding_index.get(word,None)
    if vec is not None:
        embedding_matrix[i]=vec
        
        


In [0]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable=False
del(embedding_index)
del(embedding_matrix)
del(lines)
del(counterOriginal)
del(counterTarget)

In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoderInput (InputLayer)       [(None, 50)]         0                                            
__________________________________________________________________________________________________
decoderInput (InputLayer)       [(None, 50)]         0                                            
__________________________________________________________________________________________________
Encoder_Embedding (Embedding)   (None, 50, 300)      3600900     encoderInput[0][0]               
__________________________________________________________________________________________________
Decoder_Embedding (Embedding)   (None, 50, 300)      3600900     decoderInput[0][0]               
______________________________________________________________________________________________

In [0]:
callback=keras.callbacks.ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/Work/Machine Translation/NMT_VanillaGRU.hdf5',save_weights_only=True,monitor='val_loss')

In [0]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/Work/Machine Translation/NMT_VanillaGRU.hdf5')

In [0]:
model.fit_generator(train_generator,steps_per_epoch=len(target)//batch_size,epochs=10,callbacks=[callback],verbose=True,validation_data=val_generator,
                    validation_steps=len(targetEval),shuffle=False,initial_epoch=3)

Epoch 4/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
8906/8906 [==============================] - 3746s 421ms/step - loss: 1.3191 - acc: 0.7592 - val_loss: 1.2282 - val_acc: 0.7753
Epoch 5/10
8906/8906 [==============================] - 3751s 421ms/step - loss: 1.2463 - acc: 0.7651 - val_loss: 1.2040 - val_acc: 0.7776
Epoch 6/10
8906/8906 [==============================] - 3740s 420ms/step - loss: 1.2047 - acc: 0.7687 - val_loss: 1.1896 - val_acc: 0.7795
Epoch 7/10
8906/8906 [==============================] - 3701s 416ms/step - loss: 1.1744 - acc: 0.7716 - val_loss: 1.1809 - val_acc: 0.7811
Epoch 8/10
8906/8906 [==============================] - 3682s 413ms/step - loss: 1.1516 - acc: 0.7736 - val_loss: 1.1748 - val_acc: 0.7811
Epoch 9/10
8906/8906 [==============================] - 3682s 413ms/step - loss: 1.1331 - acc: 0.7755 - val_loss: 1.1750 - val_acc: 0.7814
Epoch 10/10
8906/8906 [==============================] - 3676s 413ms/ste

In [0]:
batch_size=24
train_generator=generator(original,target,max_length_target,batch_size,vocab_target_size)
val_generator=generator(originalEval,targetEval,max_length_target,batch_size,vocab_target_size)
model.fit_generator(train_generator,steps_per_epoch=len(target)//batch_size,epochs=10,callbacks=[callback],verbose=True,validation_data=val_generator,
                    validation_steps=len(targetEval),shuffle=False)

In [0]:
targetTokenizer.index_word[0]='unk'

In [137]:
inp1=(originalEval[99])[::-1].reshape(1,-1)
inp2='<start>'
pred=[targetTokenizer.word_index[inp2]]
text=""
inp1=encoder_model.predict([inp1])
for i in range(max_length_target):
    seq=[w for w in pred]
    seq=keras.preprocessing.sequence.pad_sequences([seq],maxlen=max_length_target)
    y_hat,inp1=decoder_model.predict([inp1,seq])
    y_hat=np.argmax(y_hat)
    pred.append(y_hat)
    text=text+ " " + targetTokenizer.index_word[y_hat]
    if targetTokenizer.index_word[y_hat]=='<end>':
        break
    decoder_model.reset_states()
print(text)

 всоонк продолжали осуществлять наблюдение за unk и unk оружия <end>


In [138]:
print(" ".join(originalTokenizer.index_word[w] for w in originalEval[99] if w!=0))

<start> undof conducted unk inspections of armament and force levels in the areas of limitation <end>


In [139]:

print(" ".join(targetTokenizer.index_word[w] for w in targetEval[99] if w!=0))

<start> сооннр проводили раз в две недели проверку уровней вооружений и вооруженных сил в районах ограничения <end>


In [0]:
len(targetEval[321])